In [6]:
! pip install openpyxl gensim==3.8.3 fasttext lightgbm multipledispatch razdel

In [2]:
! wget http://vectors.nlpl.eu/repository/20/187.zip

In [3]:
! unzip 187.zip

In [4]:
! ls

In [7]:
import pandas as pd
import numpy as np
import fasttext
import razdel
import gensim

import fasttext
from gensim.models import FastText

from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, average_precision_score
from sklearn.model_selection import StratifiedShuffleSplit
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
import joblib

from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

In [8]:
columns = ['Общее наименование продукции', 'Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции)', 'Подкатегория продукции']
big = pd.read_excel('/kaggle/input/hackaton/big.xlsx', sheet_name='все ДС с кодами')[columns]
small = pd.read_excel('/kaggle/input/hackaton/small.xlsx', sheet_name='Данные для сопоставления')[columns]
big = big.rename(columns={"Общее наименование продукции": "product_name", 
                   "Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции)": "level",
                   "Подкатегория продукции": "category"})

small = small.rename(columns={"Общее наименование продукции": "product_name", 
                   "Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции)": "level",
                   "Подкатегория продукции": "category"})

all_data = pd.concat([big, small])

In [9]:
all_data['level_2'] = all_data['level'].str.split(';')
all_data['category'] = all_data['category'].str.split(';')

In [10]:
all_data = all_data.set_index(['product_name', 'level']).explode(['level_2', 'category']).reset_index()
all_data['level_1'] = all_data['level_2'].apply(lambda x: x.split('.')[0]).str.strip()
all_data['level_2'] = all_data['level_2'].str.strip()
all_data['category'] = all_data['category'].str.strip()

In [11]:
all_data.shape

In [12]:
dictionary = all_data[['category', 'level_1', 'level_2']].drop_duplicates()

In [13]:
dictionary.shape

In [14]:
dictionary.to_csv('dictionary_level_2.csv', sep=';', index=None)

In [15]:
all_data.shape

## Предобработка

In [16]:
import re
russian_stopwords = open('/kaggle/input/hackaton/stopwords-ru.txt', 'r').read().split('\n')
okpd = pd.read_excel('/kaggle/input/hackaton/okpd.xlsx').drop_duplicates(subset=['okpd'])
okpd = okpd[okpd['okpd'].str.len()>10].reset_index(drop=True).reset_index()
dictionary = pd.read_csv('/kaggle/input/hackaton/levels.csv', sep=';', encoding='cp1251').dropna(subset=['level_2'])
dictionary['level'] = dictionary['level_1'].astype(str) + '.' + dictionary['level_2'].astype(int).astype(str)
dictionary = dictionary.drop_duplicates(subset='level')

def delete_stopwords(s):
    return ' '.join([word for word in (re.sub(r'[()\s+]', u' ', s)).split() if word.lower() not in russian_stopwords]).split()

def delete_punctuation(s):
    return re.sub(r'[®?"\'-_/.:?!1234567890()%<>;,+#$&№\s+]', u' ', s)

def get_okpd(line) -> int :
    okpd_re = re.compile('окпд2\x20*(\d{2}\.\d{2}\.\d{2}\.\d{3})')
    res = re.findall(okpd_re, line.lower())
    return res[0] if len(res) > 0 else None 

In [17]:
all_data['okpd'] = all_data['product_name'].apply(get_okpd)
all_data = all_data.join(okpd.set_index('okpd'), on = 'okpd')
all_data['clean_product_name'] = all_data['product_name'].apply(lambda x: ' '.join(delete_stopwords(delete_punctuation((x)))))

In [18]:
all_data.columns

## Определение категории статистикой

In [19]:
all_categories = all_data.groupby(['level_2', 'category'])['level'].count().reset_index()
all_categories = all_categories.rename(columns={'level':'count'})

In [20]:
def get_category_sim(product_name:str, category: str, all_categories):
    all_categories['sim'] = get_similarity(product_name, all_categories)
    all_categories = all_categories.sort_values(by=['sim', 'count'], ascending=False)
    label, probability = all_categories[['level_2', 'sim']].values[0]
    return label, probability, category==label

def get_similarity(product_name:str, all_categories):
    probability = []
    for category in all_categories:
        probability.append(fuzz.token_sort_ratio(short_rp_name, category)/100)
    return probability#all_categories['category'].apply(lambda x: ).values.tolist()

In [21]:
# all_categories_list = all_categories[all_categories['count']<50]['category'].tolist()
# print(len(all_categories_list))
# all_data['features'] = all_data['clean_product_name'].progress_apply(lambda x: get_similarity(x, all_categories_list ))

## FastText обученный

In [22]:
import razdel
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


def tokenize_with_razdel(text):
    tokens = [token.text for token in razdel.tokenize(text)]
    
    return tokens

In [23]:
model = gensim.models.KeyedVectors.load("model.model")
# model =  gensim.models.FastText.load_fasttext_format('/kaggle/input/fasttext-1/fb_model.bin')

In [24]:
all_data_tokens = np.array([tokenize_with_razdel(text) for text in all_data['clean_product_name'].str.lower()])

In [25]:
# from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(analyzer=lambda x: x)
tfidf = tfidf_vectorizer.fit(all_data_tokens)
word2weight = {word : tfidf.idf_[i] for word, i in tfidf.vocabulary_.items()}

In [26]:
# Если встретится слово, которое неизвестно модели tf-idf, то буду считать его редким  и дам ему максимальный idf

max_idf = max(tfidf.idf_)
# np.array([model[token.text.lower()]*word2weight[token.text.lower()] 

In [27]:
max_idf

In [28]:
def get_text_embedding_with_tf_idf(model, phrase):
    # построение эмбеддинга для предложения
    tokens = tokenize_with_razdel(phrase)[:15]
    embedding_word = np.array([model[token.lower()]*word2weight[token.lower()] 
                               if token.lower() in word2weight 
                               else model[token.lower()]*max_idf for token in tokens])
   
    embedding_mean = embedding_word.mean(axis=0)
    if np.any(np.isnan(embedding_mean)):
        return  np.zeros((1, model.vector_size))
    else:
        return embedding_mean

In [29]:
target_labels = dict(enumerate(set(all_data['level_2'].tolist())))
labels_target = {v: k for k, v in target_labels.items()}

In [30]:
import numpy as np

y = all_data['level_2'].apply(lambda x: labels_target[x]).to_numpy()

In [31]:
X = np.zeros((all_data.shape[0], model.vector_size))
for i, embedding in enumerate(all_data["clean_product_name"].reset_index(drop=True).values):
    X[i, :] = get_text_embedding_with_tf_idf(model, embedding)

# X = np.array([get_text_embedding_with_tf_idf(model, text) for text in all_data["clean_product_name"].reset_index(drop=True).values])

print(X.shape)
print(y)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

# clf = SVC(gamma ='auto', )
clf = LinearSVC(random_state=42)

clf.fit(X_train, y_train)

In [ ]:
predictions = clf.predict(X_test)
    
print(classification_report(y_test, predictions))

## Предсказание модели  

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
from lightgbm import LGBMClassifier
import joblib

down sampling

In [ ]:
cnts = all_data['level_2'].value_counts()
normal_categories = cnts[cnts>10].index.tolist()

all_data = all_data[all_data['level_2'].isin(normal_categories)]
all_data = all_data.dropna(subset=['level_2'])
# all_data_filtred = pd.concat([ all_data[all_data.level_1==9300].drop_duplicates().sample(n=200000, replace=True, random_state=42),
#                                all_data[all_data.level_1!=9300]])

In [ ]:
train, test = train_test_split(all_data[['clean_product_name', 'level_2']], test_size=0.3, random_state=42)
train, val = train_test_split(train[['clean_product_name', 'level_2']], test_size=0.15, random_state=42)

In [ ]:
ft_model = fasttext.load_model('/kaggle/working/fb_model_deduplicated.bin')

In [ ]:
%%time
train_vectors = np.array([ft_model.get_sentence_vector(text) for text in train['clean_product_name'].str.replace('\n', ' ').str.lower()])
val_vectors = np.array([ft_model.get_sentence_vector(text) for text in val['clean_product_name'].str.replace('\n', ' ').str.lower()])
test_vectors = np.array([ft_model.get_sentence_vector(text) for text in test['clean_product_name'].str.replace('\n', ' ').str.lower()])

In [ ]:
lgbm_params = {
    'n_estimators': 300,
    'max_depth': 10,
    'learning_rate': 0.05,
    'n_jobs': 7,
    'random_state':42,
    'first_metric_only':True,
    'is_unbalance':True
}

model = LGBMClassifier(**lgbm_params)
model.fit(train_vectors, train['level_2'], verbose=True, eval_set=(val_vectors, val['level_2']),  early_stopping_rounds = 15)

In [ ]:
from sklearn import metrics

y_predicted = model.predict(test_vectors)
print(metrics.classification_report(test['level_2'], y_predicted))

In [ ]:
y_predicted = model.predict(test_vectors)
print(metrics.classification_report(test['level_1'], y_predicted))

In [ ]:
joblib.dump(model, 'model.pkl')

## Предсказание для сервиса

In [ ]:
import fasttext
from lightgbm import LGBMClassifier
import joblib
from multipledispatch import dispatch

class ProductNameClassifier:
    def __init__(self):
        self.ft_model = fasttext.load_model('/kaggle/input/01-fasttext-prototype/fb_model.bin')
        self.model = joblib.load('model.pkl')
    
    @dispatch(str)
    def predict(self, text:str) -> (int, float):
        """
        Функция для обработки текста 
        """
        text = text.replace('\n', ' ')
        text_vector = self.ft_model.get_sentence_vector(text).reshape(1, -1)
        label = int(self.model.predict(text_vector)[0])
        probability = self.model.predict_proba(text_vector)
        return label, probability.max()  
    
    @dispatch(str, int)
    def predict(self, text:str, user_label:int) -> (str, float, int):
        """
        Функция для обработки текста и метки 
        """
        text = text.replace('\n', ' ')
        text_vector = self.ft_model.get_sentence_vector(text).reshape(1, -1)
        label = self.model.predict(text_vector)[0]
        probability = self.model.predict_proba(text_vector)
        is_equal = label == user_label
        return label, probability.max(), is_equal

In [ ]:
pd.DataFrame.from_dict({'product_name': {0: 'Парацетамол таблетки 500 мг 10 шт., упаковки ячейковые контурные (2), пачки картонные, рег № ЛС-001364 от 06.08.2010, серия 190618, партия 59110 упаковок, годен до 01.07.2022, производства  ОАО "Фармстандарт-Лексредства", ИНН 4631002737, 305022, Курская область, Курск, ул. 2-я Агрегатная, 1А/18, Россия, код ОКПД2 21.20.10.232 ',
  1: 'Перезаряжаемая литий-ионная батарея торговой марки HUAWEI модель HB642735ECW',
  2: 'Перезаряжаемая литий-ионная батарея торговой марки vivo модель B-E8',
  3: 'Аппарат вакуумно-лазерной терапии стоматологический АВЛТ-"ДЕСНА" (по методу Кулаженко-Лепилина)',
  4: 'Блоки оконные и балконные дверные из алюминиевых профилей системы "INICIAL" серии "IW63" фирмы ООО "Урало-Сибирская профильная компания"'},
 'category_1': {0: 9300, 1: 3482, 2: 3482, 3: 9444, 4: 5270},
 'model_label': {0: 9300.0, 1: 3482.0, 2: 3482.0, 3: 9441.0, 4: 5772.0},
 'probability': {0: 0.6942050498562966,
  1: 0.9989457655246328,
  2: 0.9989539612756103,
  3: 0.703270209758987,
  4: 0.9999890387091874}})

In [ ]:
product_name_clf = ProductNameClassifier()

#если есть и метка от пользователя и текст
label, probability, is_equal = product_name_clf.predict(big['product_name'][0], 9300)
print(label, probability, is_equal )

#если только текст
label, probability = product_name_clf.predict(big['product_name'][0])
print(label, probability)

#Скоринг всего файла с наличием метки от пользователя
df[['model_label', 'probability', 'is_equal']] = None
df[['model_label', 'probability', 'is_equal']] = df[['product_name', 'category_1']].apply(lambda x: product_name_clf.predict(*x),
                                                                                                    axis=1,
                                                                                                    result_type="expand")

#Скоринг всего файла без метки от пользователя
df[['model_label', 'probability']] = None
df[['model_label', 'probability']] = df[['product_name']].apply(lambda x: product_name_clf.predict(*x),
                                                                                                    axis=1,
                                                                                                    result_type="expand")

In [ ]:
multipledispatch==0.6.0
lightgbm==3.2.1
joblib==1.0.1
fasttext==0.9.2
pandas==1.3.4
numpy==1.19.5
scipy==1.7.1
scikit-learn==0.23.2

In [ ]:
label, probability

In [ ]:
pip list |grep scipy

In [ ]:
df = pd.DataFrame.from_dict(
    {'product_name': {0: 'Парацетамол таблетки 500 мг 10 шт., упаковки ячейковые контурные (2), пачки картонные, рег № ЛС-001364 от 06.08.2010, серия 190618, партия 59110 упаковок, годен до 01.07.2022, производства  ОАО "Фармстандарт-Лексредства", ИНН 4631002737, 305022, Курская область, Курск, ул. 2-я Агрегатная, 1А/18, Россия, код ОКПД2 21.20.10.232 ',
      1: 'Перезаряжаемая литий-ионная батарея торговой марки HUAWEI модель HB642735ECW',
      2: 'Перезаряжаемая литий-ионная батарея торговой марки vivo модель B-E8',
      3: 'Аппарат вакуумно-лазерной терапии стоматологический АВЛТ-"ДЕСНА" (по методу Кулаженко-Лепилина)',
      4: 'Блоки оконные и балконные дверные из алюминиевых профилей системы "INICIAL" серии "IW63" фирмы ООО "Урало-Сибирская профильная компания"'},
     'category_1': {0: 9300, 1: 3482, 2: 3482, 3: 9444, 4: 5270}
    })

product_name_clf = ProductNameClassifier()


#Скоринг всего файла без метки от пользователя
df[['model_label', 'probability']] = None
df[['model_label', 'probability']] = df[['product_name']].apply(lambda x: product_name_clf.predict(*x),
                                                                                                    axis=1,
                                                                                                    result_type="expand")
print('3:\n', df.head())
#Скоринг всего файла с наличием метки от пользователя
df[['model_label', 'probability', 'is_equal']] = None
df[['model_label', 'probability', 'is_equal']] = df[['product_name', 'category_1']].apply(lambda x: product_name_clf.predict(*x),
                                                                                                    axis=1,
                                                                                                    result_type="expand")
print('3:\n', df.head())